* # We need to know when how electoral divisions have changed
* # Are they all in County and City council records or where?
* # When former districts are refered to is it the $1925$ districts, the pool law districts? ...

# Preparations
Undoubtedly there are python libraries that will succussfully extract the text from the schedule. I didn't find them or perhaps just didn't find the commands to operate them successfully
## tabula
java package kind of working
## ghostscript
here is a blast from the past
## pdftk
not in repo; needs gcj
>## docker
There are probably a few ways to drive pdftk without gcj running natively. I took the opportunity to use docker.io, as much as anything, just so I could say I had used it. 


[Markdown cheetsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

```bash
java -jar tabula.jar -n -p all -f TSV -o '2017-12-23--39-electoral-amendment-dail-cons.pdf' '2017-NEW.csv'
```

# tabula
## why tabula
* There is a gui and an api so we can tune the extraction visually.
* The gui automatically grabbed only the table; some tools grabbed everything.

Follow the instructions for [tabula](https://github.com/tabulapdf/tabula).

Run the GUI and open the browser at [http://127.0.0.1:8080/]("")
```bash
foo@bar:~$ $ java -jar tabula.jar
```

Auto-detect tables. Observe the highlighted areas cover the table and only the table. Press 'Preview and export extracted data'. Doesn't look great. On left press 'Lattice'. Looks much better but lots of the table is missing. The 'Lattice' option looks for boundary lines. Notice that there are no boundary lines at the bottom of the pages and that the missing pieces of the table match the final, unclosed cell on each page.

We could maybe try to find or create an option in tabula to include the unclosed cells or we could close the cells and see if that works. 

Let's try closing the cells by stamping a horizontal rule at the bottom of each page.

# ghostscript
We us ghostscript to create a A4 size .pdf file that we'll use to stamp on the act.

We inspect the tabula tempelate and see ...

```console
:~$ gs -o line.pdf  -sDEVICE=pdfwrite -g5950x8420  -sPAPERSIZE=a4  -c "newpath 84 126 moveto 510 126 lineto 0 setgray gsave 2 setlinewidth stroke showpage"
```

# pdftk
[examples](https://www.pdflabs.com/docs/pdftk-cli-examples/)

[man-page](https://www.pdflabs.com/docs/pdftk-man-page/)

pdftk requires gcj and neither are in the debian repos any longer; let's look into using docker.
> ## docker
should really find the link for the instructions I followed...
```console
:~$ sudo apt install docker.io
:~$ sudo docker run -it ubuntu:16.04 bash
```
follow post install instructions [here](https://docs.docker.com/install/linux/linux-postinstall/)

> In the docker shell:
>>```console
:~# apt update
:~# apt install pdftk
```

>```console
:~$ sudo docker ps -a
:~$ sudo docker commit #your_ID ubuntu_pdftk
```
```console
:~$ sudo touch /usr/bin/pdftk
:~$ sudo nano /usr/bin/pdftk
```
>> ```console
#!/bin/sh
set -eu
docker run --name pdftk -it -v "$PWD:/workdir$PWD" -w "/workdir$PWD" ubuntu_pdftk pdftk $
docker rm pdftk
```
#### notice that the -it flag must be removed to run the command from python 

>```console
:~$ sudo chmod +x /usr/bin/pdftk
```


With pdftk operational
```console
:~$ sudo pdftk Act2017.pdf stamp line.pdf output Act2017_stamped.pdf
```
I <emph>guess</emph> we need the sudo here to sun the docker instance

Notice that the horizontal rule is at the same height on each page while the tables do not terminate at equal heights. We can use the template function from tabula to get the terminating heights.


## notes:
see what is running on docker

```console
:~$ docker ps -a
```

kill a running process in the docker shell
```console
:~$ docker rm pdftk
``` 

# A few more steps

In [1]:
# page by page stamping
import pandas as pd
import os


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 56, got 52
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 56, got 52
  return f(*args, **kwds)


In [2]:

pgs = pd.read_json('2017_tabula_guesses.json')


In [3]:
pgs.columns

Index(['extraction_method', 'height', 'page', 'width', 'x1', 'x2', 'y1', 'y2'], dtype='object')

In [88]:
import subprocess

for page, height in zip(pgs.page, pgs.y1):
#     print(page, height)
#     omg too much time spent messing with subprocess
    os.system(f'gs -o {page}.pdf  -sDEVICE=pdfwrite -g5950x8420  -sPAPERSIZE=a4  -c "newpath 84 {height} moveto 510 {height} lineto 0 setgray gsave 2 setlinewidth stroke showpage"')
    os.system(f'sudo pdftk Act2017.pdf stamp {page}.pdf output Act2017.pdf')

In [82]:
h = 700
os.system(f'gs -o shit.pdf   -sDEVICE=pdfwrite -c "newpath 84 {h} moveto 510 {h} lineto gsave stroke"')

0

In [11]:
os.system(f'pdftk A=blank.pdf cat A A A A A output tmp.pdf')

# A = 'blank.pdf'
# os.sysetm('cp blank.pdf tmp.pdf')
# weird, following will not clobber if file exists
# for i in range(1, pgs.iloc[0].page -1):
#     os.system(f'pdftk A=blank.pdf cat A A A A A output tmp.pdf')
#     print(i)

1
2
3
4
5


In [19]:
for i in pgs.page:
    print(i)    
    os.system(f'pdftk A=tmp.pdf B={i}.pdf cat A B output tmp1.pdf')
    os.system('mv tmp1.pdf tmp.pdf')

7
8
9
10
11
12
13
14
15
16
17
18
19


0

### create blank A4 pdf
```console
echo "" | ps2pdf -sPAPERSIZE=a4 - blank.pdf
```
or
```console
touch blank.ps;
ps2pdf blank.ps blank.pdf;
```
### concatenate pdf fiels
1.pdf and blank.pdf already exists
```console
pdftk A=1.pdf B=blank.pdf cat A1-end B output 2.pdf
```

# So now we have a pandas data frame of each schedule as it appears on the pdf of each Act.
## I guess I can upload the csv too and link to it here


```console
foo@bar:~$ java -jar tabula.jar
```

In [6]:
# import PyTables as pt



import tabula
import os
import pandas as pd
 
folder = 'data/pdf/'
paths = [folder + fn for fn in os.listdir(folder) if fn.endswith('.pdf')]

    df = tabula.read_pdf( encoding = 'latin1', pages = 'all', area = [29.75,43.509,819.613,464.472], nospreadsheet = True)
    path = path.replace('pdf', 'csv')
    df.to_csv(path, index = False)
    

ModuleNotFoundError: No module named 'PyTables'